<a href="https://colab.research.google.com/github/DmitryKutsev/ml_hw/blob/master/hw_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/TatianaShavrina/hse_ml_m1/master/ensembles/complaints.csv

--2020-04-06 11:12:43--  https://raw.githubusercontent.com/TatianaShavrina/hse_ml_m1/master/ensembles/complaints.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4148456 (4.0M) [text/plain]
Saving to: ‘complaints.csv’

complaints.csv      100%[===================>]   3.96M  18.9MB/s    in 0.2s    

2020-04-06 11:12:44 (18.9 MB/s) - ‘complaints.csv’ saved [4148456/4148456]



In [2]:
!pip install pymorphy2[fast]

     |████████████████████████████████| 51kB 915kB/s 
     |████████████████████████████████| 7.1MB 3.1MB/s 
     |████████████████████████████████| 378kB 32.1MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp36-cp36m-linux_x86_64.whl size=862179 sha256=4cc9aa58370c0ff3032d06cfebcab56153d6c024895937c534aa9a1c55b94524
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG


In [0]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_extraction.text import *
from sklearn.metrics import *
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression, LinearRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier,  BaggingClassifier, BaggingRegressor, RandomTreesEmbedding,GradientBoostingClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk import word_tokenize
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [4]:
data = pd.read_csv('complaints.csv', sep='\t')
print(data.head(10))
y = data["PRODUCT_ID"]
X = data["cleaned_text"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

   COMPLAINT_ID  ...                                       cleaned_text
0       3178905  ...  go year . contact advis never took loan . advi...
1       3175952  ...  mail valid debt xx/xx/19 valid receiv , receiv...
2       3174747  ...  xx/xx/xxxx appli receiv onlin loan bluechip fi...
3       3173291  ...  xx/xx/xxxx appli receiv onlin loan . loan amou...
4       3172221  ...  told husband left bill . debt would pay within...
5       3172723  ...  payday loan , becam abl pay . everyon famili c...
6       3171497  ...  within past week , contact oracl financi group...
7       3171674  ...  call work said courthous owe money . got phone...
8       3170784  ...  today xx/xx/xxxx cst contact ks . receiv appro...
9       3171479  ...  cashcal attempt collect fraudul debt . ident s...

[10 rows x 5 columns]


In [0]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import FunctionTransformer

In [0]:
clf1 = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()

eclf = VotingClassifier(estimators=[
        ('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')

voting = Pipeline([
    ('vect', CountVectorizer( analyzer='word', max_features=300)),
    ('tfidf', TfidfTransformer(sublinear_tf=True)),
    ('to_dense', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)), 
    ('reducer', TruncatedSVD(n_components=100)),
    ('clf', eclf),
    ])
voting = voting.fit(X_train, y_train)

In [27]:
predictions = voting.predict(X_test)
print("Precision: {0:6.2f}".format(precision_score(y_test, predictions, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, predictions, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, predictions, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, predictions)))

Precision:   0.67
Recall:   0.66
F1-measure:   0.66
Accuracy:   0.66


Здесь покрутил параметры, результат вроде стал больше, чем в лекции, посмотрим дальше.